In [1]:
#! pip install PyPDF2
# ! pip install langchain-community langchain-core
#! pip install openai


In [2]:
import base64
from langchain_community.chat_models import AzureChatOpenAI
import os
from IPython.display import Image, display
import base64
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

images_summarizer_prompt = """
As an analyst for company, please provide a clear interpretation of data or information that describes the image.
Describe the image and its key elements. Try to provide explaination of the image in a way that is easy to understand for a non-technical audience.
Highlight significant data points or trends that are relevant.
Avoid bullet points; instead, deliver a coherent, factual summary that captures the essence of the image for decision-making.
Limit your description to 3-4 sentences, ensuring it's precise and informative for analysis. Respond in Markdown format."""

tables_summarizer_prompt = """ As an analyst expert at summarizing tables, provide a concise and exact summary of the table contents. Highlight significant trends, comparisons, or outliers in the data. Avoid bullet points; instead, deliver a coherent, factual summary that captures the essence of the table.
Limit your summary to 3-4 sentences, ensuring it's precise and informative for analysis and also efficient for performing vector search. Respond in Markdown format."""

attribute_summarizer_prompt = """
As an analyst expert at Extract the following details from input context.
Highlight job no, Part Number or PN, Revision Number, SN or Serial Number, Date of analysis, Name of the engineers who is performing the analysis, Name of the engineer to whom the analysis was submitted, Subject of the report and finally which Company performing the analysis.
Limit your summary to 3-4 sentences, ensuring it's precise and informative for analysis and also efficient for performing vector search. Respond in key, value type in markdown format.
"""

text_summarizer_prompt = """ As an analyst expert at summarizing context, provide a concise and exact summary of the input contexts. And Summarize the key findings from this context.
Limit your summary to 3-4 sentences, ensuring it's precise and informative for analysis and also efficient for performing vector search. Respond in Markdown format."""



#IMAGE_PATH = "C:/Users/Nadigsh/Downloads/compute instance error.png"

os.environ["OPENAI_API_KEY"] = "c866d334576a443db7bd0f8f09d81990"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://dfapidmdev-gw.lamrc.net/"

llm = AzureChatOpenAI(
    openai_api_key="c866d334576a443db7bd0f8f09d81990",
    azure_endpoint = "https://dfapidmdev-gw.lamrc.net/",
    openai_api_version= "2023-07-01-preview",
    deployment_name="gpt-4o",
    temperature=0,
    streaming=True,
    client="Azure",
    callbacks=[StreamingStdOutCallbackHandler()]  # Allows real-time processing of output from OpenAI API
)

# Open the image file and encode it as a base64 string
def encode_image(image_path):
   with open(image_path, "rb") as image_file:
       return base64.b64encode(image_file.read()).decode("utf-8")

def extract_image_summary(image_path, images_summarizer_prompt):
    print("Passed Image: ") # Preview image for context
    display(Image(image_path))

    base64_image = encode_image(image_path)
    messages=[
       {"role": "system", "content": images_summarizer_prompt},
       {"role": "user", "content": [
           {"type": "image_url", "image_url": {
               "url": f"data:image/png;base64,{base64_image}"}
           }
       ]}
    ]
    ai_message = llm.invoke(messages)
    #print(ai_message.content)
    return ai_message.content

def extract_imageSummary_with_docSummary(image_path, images_summarizer_prompt, docsummary_data):
    #print("Passed Image: ") # Preview image for context
    #display(Image(image_path))

    base64_image = encode_image(image_path)
    messages=[
       {"role": "system", "content": images_summarizer_prompt},
       {"role": "user", "content": [
           {"type": "text", 
            "text": f"Describe this image (note: it has document summary data: {docsummary_data}):" 
            },
           {"type": "image_url", "image_url": {
               "url": f"data:image/png;base64,{base64_image}"}
           }
       ]}
    ]
    ai_message = llm.invoke(messages)
    #print(ai_message.content)
    return ai_message.content

def extract_table_summary(table_content, tables_summarizer_prompt):
    messages=[
       {"role": "system", "content": tables_summarizer_prompt},
       {"role": "user", "content": [
           {"type": "text", "text": table_content}
       ]}
    ]
    ai_message = llm.invoke(messages)
    return ai_message.content

def extract_attribute_summary(text_content, text_summarizer_prompt):
    messages=[
       {"role": "system", "content": attribute_summarizer_prompt},
       {"role": "user", "content": [
           {"type": "text", "text": text_content}
       ]}
    ]
    ai_message = llm.invoke(messages)
    return ai_message.content


#### Extract the first 2 pages summary of the document ####
def extract_doc_summary(text_content, text_summarizer_prompt):
    messages=[
       {"role": "system", "content": text_summarizer_prompt},
       {"role": "user", "content": [
           {"type": "text", "text": text_content}
       ]}
    ]
    ai_message = llm.invoke(messages)
    return ai_message.content


#extract_image_summary("C:/Users/Nadigsh/Downloads/compute instance error.png", images_summarizer_prompt)
#extract_table_summary("""""", tables_summarizer_prompt)

c:\Users\AshokPa1\OneDrive - Lam Research\Desktop\Projects\ESG_Multimodal_RAG\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [3]:
from pdf2image import convert_from_path

def getImagefromPdf(pdf_sourcefile):
    sample_images = convert_from_path(pdf_sourcefile)
    return sample_images

In [4]:
import PyPDF2

def extract_first_two_pages(input_pdf_path):
    pageContent = ""
    # Open the input PDF file
    with open(input_pdf_path, 'rb') as input_pdf_file:
        reader = PyPDF2.PdfReader(input_pdf_file)
        
        # Add the first two pages to the writer object
        for page_num in range(min(2, len(reader.pages))):
            page = reader.pages[page_num]
            extractContent = page.extract_text()
            #print("extractContent is : ",extractContent)
            pageContent += extractContent
    return pageContent
# Example usage
#input_pdf_path = './data/245-21-1174A.pdf'

#extract_first_two_pages(input_pdf_path)

In [5]:
import os
from dotenv import load_dotenv
import requests

# Load environment variables
#load_dotenv()
# os.getenv("VISION_ENDPOINT") = "https://compvision-lam-usw-dev-001.cognitiveservices.azure.com/"
# os.getenv("VISION_KEY") = "59dcf2ab5f2445c9b9db0b74b8614e3f"
os.environ["VISION_ENDPOINT"] = "https://compvision-lam-usw-dev-001.cognitiveservices.azure.com/"
os.environ["VISION_KEY"] = "59dcf2ab5f2445c9b9db0b74b8614e3f"

endpoint = "https://compvision-lam-usw-dev-001.cognitiveservices.azure.com/" + "computervision/"
key = "59dcf2ab5f2445c9b9db0b74b8614e3f"

def get_image_embedding(image):
    with open(image, "rb") as img:
        data = img.read()

    # Vectorize Image API
    version = "?api-version=2023-02-01-preview&modelVersion=latest"
    vectorize_img_url = endpoint + "retrieval:vectorizeImage" + version

    headers = {
        "Content-type": "application/octet-stream",
        "Ocp-Apim-Subscription-Key": key
    }

    try:
        r = requests.post(vectorize_img_url, data=data, headers=headers)

        if r.status_code == 200:
            image_vector = r.json()["vector"]
            return image_vector
        else:
            print(f"An error occurred while processing {image}. Error code: {r.status_code}.")

    except Exception as e:
        print(f"An error occurred while processing {image}: {e}")

    return None

# image_filename = "images/image (1).jpg"
# image_vector = get_image_embedding(image_filename)

In [9]:
def contextClean(summary_data):
    cleanedSummary = summary_data.replace("```","")
    cleanedSummary = cleanedSummary.replace("markdown","")
    return cleanedSummary

In [ ]:
import json

def get_text_embedding(prompt):
    text = {'text': prompt}

    # Image retrieval API
    version = "?api-version=2023-02-01-preview&modelVersion=latest"
    vectorize_txt_url = endpoint + "retrieval:vectorizeText" + version

    headers = {
        'Content-type': 'application/json',
        'Ocp-Apim-Subscription-Key': key
    }

    try:
        r = requests.post(vectorize_txt_url, data=json.dumps(text), headers=headers)

        if r.status_code == 200:
            text_vector = r.json()['vector']
            return text_vector
        else:
            print(f"An error occurred while processing the prompt '{text}'. Error code: {r.status_code}.")

    except Exception as e:
        print(f"An error occurred while processing the prompt '{text}': {e}")

    return None

# text_prompt = "a blue house"
# text_vector = get_text_embedding(text_prompt)

In [6]:
input_pdf_path = './data/245-22-1250B Report.pdf'
All_imageChunks = []

textcontent = extract_first_two_pages(input_pdf_path)

attributeSummary_output = extract_attribute_summary(textcontent, attribute_summarizer_prompt)
attributeSummary = contextClean(attributeSummary_output)
docSummary_output = extract_doc_summary(textcontent, text_summarizer_prompt)
docSummary = contextClean(docSummary_output)
# attributeSummary_output = attributeSummary_output.replace("```","")
# attributeSummary_output = attributeSummary_output.replace("markdown","")
# docSummary_output = docSummary_output.replace("```","")
# docSummary_output = docSummary_output.replace("markdown","")
# print(type(docSummary_output))
# print("docSummary is : ",docSummary_output)
imagelist = getImagefromPdf(input_pdf_path)
getfilename = input_pdf_path.split('/')[-1]
new_filename = getfilename.split(".")[0]
print("getfilename is : ",getfilename)
for i, image in enumerate(imagelist):
    pageInfo = {}
    chunk = {}
    imagepath = f'./data/images/{new_filename}_{i+1}.png'
    pageNumber = i + 1
    image.save(imagepath, 'PNG')
    getBase64 = encode_image(imagepath)
    text_prompt = getBase64 + docSummary + attributeSummary
    image_vector = get_image_embedding(imagepath)
    text_vector = get_text_embedding(text_prompt)
    print("text_vector is : ",text_vector)
    #print("getBase64 is : ",getBase64)
    #imgSummary_output = extract_image_summary(imagepath,)
    imgSummary_output = extract_imageSummary_with_docSummary(imagepath, images_summarizer_prompt, docSummary)
    pageInfo["document_name"] = str(getfilename)
    pageInfo["page_number"] = str(pageNumber)
    chunk['document_metadata'] = str(attributeSummary)
    chunk['document_summary'] = str(docSummary)
    chunk['page_image_summary'] = str(imgSummary_output)
    pageInfo['chunk'] = chunk
    pageInfo["text_vector"] = text_vector
    pageInfo["image_vector"] = image_vector
    #pageInfo["base64encode"] = getBase64
    pageInfo["base64encode"] = ""

    # pageInfo["pdfmetadata"] = str(textSummary_output)
    # pageInfo["imagesummary"] = str(imgSummary_output)
    All_imageChunks.append(pageInfo)

print("\n")
print("all image list is : ")
print("\n")
print(All_imageChunks)




```markdown
- **Job No**: 245-22-1250B
- **Part Number or PN**: 839-269820-005 Rev. C
- **Revision Number**: C
- **SN or Serial Number**: 1035067-20-24-0005, 1035067-20-23-0007
- **Date of analysis**: January 20, 2021
- **Name of the engineers performing the analysis**: Mark A. Lisin, P.E., Jennifer McElroy
- **Name of the engineer to whom the analysis was submitted**: Nick Linebarger
- **Subject of the report**: Photographs and SEM/EDX Analysis of Two Showerheads
- **Company performing the analysis**: Lisin Metallurgical Services, LLC
``````markdown
Lisin Metallurgical Services conducted a failure analysis on two showerheads for Lam Research, focusing on interior surface staining. The analysis, which included SEM/EDX, revealed significant staining and discoloration on both the front and back interior surfaces, with elevated levels of fluorine, oxygen, and magnesium in the stained areas. The presence of mud-cracked deposits and micron-sized particles suggests residue from an evaporated

In [7]:
# import ast
# textSummary_output = textSummary_output.replace("```","")
# textSummary_output = textSummary_output.replace("json","")
# print("textSummary_output is : ",textSummary_output)
# textSummary_dict = ast.literal_eval(textSummary_output)
# print(textSummary_dict)

In [8]:
#! pip install pdf2image Pillow
#! pip install poppler-utils
#! pip install python-dotenv